In [1]:
base_path = "/home/grozin/.kaggle/competitions/avito-demand-prediction/"
train_image_base_path = base_path + "data/competition_files/train_jpg/"
test_image_base_path = base_path + "data/competition_files/test_jpg/"

In [2]:
import pickle
import pandas as pd
import numpy as np

import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Gradient Boosting
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold

# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 

# Viz
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string


/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
training = pd.read_csv(base_path + 'train.csv', index_col = "item_id", parse_dates = ["activation_date"])
testing = pd.read_csv(base_path + 'test.csv', index_col = "item_id", parse_dates = ["activation_date"])
testdex = testing.index
traindex = training.index

ntrain = training.shape[0]
ntest = testing.shape[0]

y = training.deal_probability.copy()
training.drop("deal_probability",axis=1, inplace=True)

df = pd.concat([training, testing],axis=0)
del training, testing
gc.collect()

28

In [4]:
df["price"] = np.log(df["price"]+0.001)

In [5]:
df["price"].fillna(df["price"].mean(), inplace=True)

# Features to measure distance on 

### Image vectors

In [17]:
import pickle
image_vectors_dense = {}

with open("vectors_compacted1300k.pickle", "rb") as f:
    while True:
        try:
            pair = pickle.load(f)
            image_vectors_dense[pair[0]] = pair[1]
        except EOFError:
            break

In [18]:
def norm_v(x):
    return x / np.linalg.norm(x)

col_count = len(pair[1])
zero_obj = -np.ones((col_count))
zero_obj.fill(np.nan)

dense_img_vec_names = [str(x + 1) + "_dense_img_vector" for x in range(col_count)]

packed_images_vectors = df["image"].apply(lambda x: norm_v(image_vectors_dense.get(str(x) + ".jpg", zero_obj)))
df[dense_img_vec_names] = pd.DataFrame(packed_images_vectors.values.tolist(), index=df.index)

In [19]:
del image_vectors_dense

### Tf-Idf

In [6]:
with open("tf-idf.pickle", "rb") as f:
    tf_idfs = pickle.load(f)

# Building indices

In [21]:
from sklearn.neighbors import KDTree

In [22]:
df_with_image = df[~df[dense_img_vec_names[0]].isnull()]
df_with_image_train = df_with_image.loc[df_with_image.index & traindex]
img_tree = KDTree(df_with_image_train[dense_img_vec_names], leaf_size=2)
img_targets = np.array(y[df_with_image.index])

In [23]:
cat_img_subset = {}
cat_img_subset_train = {}

cat_img_indices = {}
# cat_img_targets = {}
# cat_img_prices = {}
# cat_img_mask = {}

for cat in np.unique(df_with_image["category_name"]):
    print(cat)
    
    subset = df_with_image[df_with_image["category_name"] == cat]
    subset_train = df_with_image_train[df_with_image_train["category_name"] == cat]
    
    cat_img_subset[cat] = subset
    cat_img_subset_train[cat] = subset_train
    
    # cat_img_mask[cat] = np.where(mask)[0]
    cat_img_indices[cat] = KDTree(subset_train[dense_img_vec_names], leaf_size=2)
    # cat_img_targets[cat] = y.loc[subset.index]
    # cat_img_prices[cat] = subset["price"]

Автомобили
Аквариум
Аудио и видео
Билеты и путешествия
Бытовая техника
Велосипеды
Водный транспорт
Гаражи и машиноместа
Готовый бизнес
Грузовики и спецтехника
Детская одежда и обувь
Дома, дачи, коттеджи
Другие животные
Земельные участки
Игры, приставки и программы
Квартиры
Книги и журналы
Коллекционирование
Коммерческая недвижимость
Комнаты
Кошки
Красота и здоровье
Мебель и интерьер
Мотоциклы и мототехника
Музыкальные инструменты
Настольные компьютеры
Недвижимость за рубежом
Ноутбуки
Оборудование для бизнеса
Одежда, обувь, аксессуары
Оргтехника и расходники
Охота и рыбалка
Планшеты и электронные книги
Посуда и товары для кухни
Предложение услуг
Продукты питания
Птицы
Растения
Ремонт и строительство
Собаки
Спорт и отдых
Телефоны
Товары для детей и игрушки
Товары для животных
Товары для компьютера
Фототехника
Часы и украшения


In [7]:
from pysparnn.cluster_index import ClusterIndex

# Assuming no filtering was applied...
tf_idf_index = ClusterIndex(tf_idfs[:len(traindex)], list(range(len(traindex))))
df_tf_idf = df.loc[traindex]
tf_idf_targets = y.loc[traindex]

/usr/local/lib/python3.6/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)


# Feature utility

In [22]:
ks = [5, 10, 50, 100, 500] 
thresholds = [.00001, 0.01, 0.1, 0.2]

In [23]:
def get_features(i, train_indices, train_dists, train_prices, train_targets, prices):
    mask = np.where(train_indices == i)
    indices_clean = np.delete(train_indices, mask)
    dists_clean = np.delete(train_dists, mask)
    
    threshold_features = np.empty(len(thresholds))
    for j, t in enumerate(thresholds):
        threshold_features[j] = sum(dists_clean < t)
    
    nearest_features = np.empty((9 * len(ks)))
    for j, k in enumerate(ks):
        prices_sub = train_prices[indices_clean[:k]]
        target_sub = train_targets[indices_clean[:k]]
        weight_sub = dists_clean[:k]
        
        nearest_features[j * 9] = np.mean(prices_sub)
        nearest_features[j * 9 + 1] = np.var(prices_sub)
        nearest_features[j * 9 + 2] = np.sum(prices_sub * weight_sub) / np.sum(weight_sub)
        nearest_features[j * 9 + 3] = prices[i] - nearest_features[j * 9]
        nearest_features[j * 9 + 4] = prices[i] - nearest_features[j * 9 + 2]
        
        nearest_features[j * 9 + 5] = np.mean(target_sub)
        nearest_features[j * 9 + 6] = np.var(target_sub)
        nearest_features[j * 9 + 7] = np.sum(target_sub * weight_sub) / np.sum(weight_sub)
        
        nearest_features[j * 9 + 8] = np.mean(dists_clean)
    return np.concatenate([threshold_features, nearest_features])

In [24]:
feature_names = ["threshold_" + str(x) for x in thresholds]
for k in ks:
    feature_names.extend([x + str(k) for x in ["mean_price_", "var_price_", "wmean_price_",
                          "delta_price_", "wdelta_price_", 
                          "mean_target_", "var_target_", "wmean_target_",
                          "mean_dist_"]])

# Global features

## Image features

In [9]:
from time import time
nearest_objects = img_tree.query(np.array(df_with_image[dense_img_vec_names]), k=np.max(ks))

AttributeError: 'ClusterIndex' object has no attribute 'query'

In [ ]:
from tqdm import tqdm

train_prices = np.array(df_with_image_train["price"])
train_targets = np.array(y.loc[df_with_image_train.index])
prices = np.array(df_with_image["price"])
img_features = []

for i, (dists, indices) in enumerate(zip(*nearest_objects)):
    f = get_features(i, indices, dists,
                     train_prices, train_targets,
                     prices)
    img_features.append(f)

img_nearest_features = np.vstack(img_features)
del img_features, prices, train_prices, train_targets

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
with open("_tmp_nearest_img_features.pickle", "wb") as f:
    pickle.dump(img_nearest_features, f)

In [ ]:
with open("_tmp_nearest_img_features.pickle", "rb") as f:
    img_nearest_features = pickle.load(f)
img_nearest_features = pd.DataFrame(img_nearest_features, columns = ["img_" + x for x in feature_names],
                                    index=df_with_image.index)
img_nearest_features.to_csv("img_nearest_features.csv.zip", compression="zip")

In [ ]:
del img_nearest_features
gc.collect()

## TF-IDF

In [40]:
from tqdm import tqdm
indices = list(range(tf_idfs.shape[0]))

train_prices = np.array(df.loc[traindex, "price"])
train_targets = np.array(y.loc[traindex])
prices = np.array(df["price"])
tfidf_features = []

tf_idf_nearest = []

for batch_indices in tqdm(np.array_split(indices, tf_idfs.shape[0] / 1000)):
    tf_idf_nearest_batch = tf_idf_index.search(tf_idfs[batch_indices], k=max(ks))
    
    for i, (dist_ind) in enumerate(tf_idf_nearest_batch):
        dists = [x[0] for x in dist_ind]
        indices = [x[1] for x in dist_ind]
        f = get_features(batch_indices[i], indices, dists,
                         train_prices, train_targets,
                         prices)
        tfidf_features.append(f)

    # tf_idf_nearest.extend(tf_idf_nearest_batch)




  0%|          | 0/2011 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars



  0%|          | 1/2011 [00:06<3:21:17,  6.01s/it]


  0%|          | 2/2011 [00:11<3:15:38,  5.84s/it]/usr/local/lib/python3.6/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)



  0%|          | 3/2011 [00:17<3:13:39,  5.79s/it]


  0%|          | 4/2011 [00:23<3:12:30,  5.76s/it]


  0%|          | 5/2011 [00:28<3:12:48,  5.77s/it]


  0%|          | 6/2011 [00:34<3:12:15,  5.75s/it]


  0%|          | 7/2011 [00:40<3:12:03,  5.75s/it]


  0%|          | 8/2011 [00:46<3:12:20,  5.76s/it]


  0%|          | 9/2011 [00:51<3:12:13,  5.76s/it]


  0%|          | 10/2011 [0

  6%|▌         | 122/2011 [11:48<3:02:48,  5.81s/it]


  6%|▌         | 123/2011 [11:54<3:02:42,  5.81s/it]


  6%|▌         | 124/2011 [12:00<3:02:39,  5.81s/it]


  6%|▌         | 125/2011 [12:06<3:02:37,  5.81s/it]


  6%|▋         | 126/2011 [12:11<3:02:29,  5.81s/it]


  6%|▋         | 127/2011 [12:17<3:02:22,  5.81s/it]


  6%|▋         | 128/2011 [12:23<3:02:14,  5.81s/it]


  6%|▋         | 129/2011 [12:29<3:02:08,  5.81s/it]


  6%|▋         | 130/2011 [12:34<3:02:03,  5.81s/it]


  7%|▋         | 131/2011 [12:40<3:01:57,  5.81s/it]


  7%|▋         | 132/2011 [12:46<3:01:49,  5.81s/it]


  7%|▋         | 133/2011 [12:52<3:01:44,  5.81s/it]


  7%|▋         | 134/2011 [12:57<3:01:37,  5.81s/it]


  7%|▋         | 135/2011 [13:03<3:01:33,  5.81s/it]


  7%|▋         | 136/2011 [13:09<3:01:25,  5.81s/it]


  7%|▋         | 137/2011 [13:15<3:01:20,  5.81s/it]


  7%|▋         | 138/2011 [13:21<3:01:13,  5.81s/it]


  7%|▋         | 139/2011 [13:26<3:01:07,  5.81s/it]


  7%|▋    

 12%|█▏        | 250/2011 [24:18<2:51:13,  5.83s/it]


 12%|█▏        | 251/2011 [24:24<2:51:07,  5.83s/it]


 13%|█▎        | 252/2011 [24:30<2:51:02,  5.83s/it]


 13%|█▎        | 253/2011 [24:36<2:50:58,  5.84s/it]


 13%|█▎        | 254/2011 [24:42<2:50:52,  5.83s/it]


 13%|█▎        | 255/2011 [24:48<2:50:46,  5.84s/it]


 13%|█▎        | 256/2011 [24:53<2:50:41,  5.84s/it]


 13%|█▎        | 257/2011 [24:59<2:50:36,  5.84s/it]


 13%|█▎        | 258/2011 [25:05<2:50:30,  5.84s/it]


 13%|█▎        | 259/2011 [25:11<2:50:25,  5.84s/it]


 13%|█▎        | 260/2011 [25:17<2:50:19,  5.84s/it]


 13%|█▎        | 261/2011 [25:23<2:50:14,  5.84s/it]


 13%|█▎        | 262/2011 [25:29<2:50:08,  5.84s/it]


 13%|█▎        | 263/2011 [25:35<2:50:02,  5.84s/it]


 13%|█▎        | 264/2011 [25:40<2:49:56,  5.84s/it]


 13%|█▎        | 265/2011 [25:46<2:49:50,  5.84s/it]


 13%|█▎        | 266/2011 [25:52<2:49:45,  5.84s/it]


 13%|█▎        | 267/2011 [25:58<2:49:39,  5.84s/it]


 13%|█▎   

 19%|█▉        | 378/2011 [36:50<2:39:08,  5.85s/it]


 19%|█▉        | 379/2011 [36:55<2:39:02,  5.85s/it]


 19%|█▉        | 380/2011 [37:01<2:38:56,  5.85s/it]


 19%|█▉        | 381/2011 [37:07<2:38:50,  5.85s/it]


 19%|█▉        | 382/2011 [37:13<2:38:44,  5.85s/it]


 19%|█▉        | 383/2011 [37:19<2:38:39,  5.85s/it]


 19%|█▉        | 384/2011 [37:25<2:38:33,  5.85s/it]


 19%|█▉        | 385/2011 [37:31<2:38:27,  5.85s/it]


 19%|█▉        | 386/2011 [37:37<2:38:22,  5.85s/it]


 19%|█▉        | 387/2011 [37:43<2:38:16,  5.85s/it]


 19%|█▉        | 388/2011 [37:48<2:38:11,  5.85s/it]


 19%|█▉        | 389/2011 [37:54<2:38:05,  5.85s/it]


 19%|█▉        | 390/2011 [38:00<2:37:59,  5.85s/it]


 19%|█▉        | 391/2011 [38:06<2:37:54,  5.85s/it]


 19%|█▉        | 392/2011 [38:12<2:37:48,  5.85s/it]


 20%|█▉        | 393/2011 [38:18<2:37:43,  5.85s/it]


 20%|█▉        | 394/2011 [38:24<2:37:39,  5.85s/it]


 20%|█▉        | 395/2011 [38:30<2:37:34,  5.85s/it]


 20%|█▉   

 25%|██▌       | 506/2011 [49:24<2:26:56,  5.86s/it]


 25%|██▌       | 507/2011 [49:29<2:26:50,  5.86s/it]


 25%|██▌       | 508/2011 [49:35<2:26:44,  5.86s/it]


 25%|██▌       | 509/2011 [49:41<2:26:39,  5.86s/it]


 25%|██▌       | 510/2011 [49:47<2:26:33,  5.86s/it]


 25%|██▌       | 511/2011 [49:53<2:26:27,  5.86s/it]


 25%|██▌       | 512/2011 [49:59<2:26:21,  5.86s/it]


 26%|██▌       | 513/2011 [50:05<2:26:15,  5.86s/it]


 26%|██▌       | 514/2011 [50:11<2:26:09,  5.86s/it]


 26%|██▌       | 515/2011 [50:17<2:26:04,  5.86s/it]


 26%|██▌       | 516/2011 [50:22<2:25:58,  5.86s/it]


 26%|██▌       | 517/2011 [50:28<2:25:52,  5.86s/it]


 26%|██▌       | 518/2011 [50:34<2:25:46,  5.86s/it]


 26%|██▌       | 519/2011 [50:40<2:25:41,  5.86s/it]


 26%|██▌       | 520/2011 [50:46<2:25:35,  5.86s/it]


 26%|██▌       | 521/2011 [50:52<2:25:29,  5.86s/it]


 26%|██▌       | 522/2011 [50:58<2:25:23,  5.86s/it]


 26%|██▌       | 523/2011 [51:04<2:25:17,  5.86s/it]


 26%|██▌  

...

 74%|███████▍  | 1496/2011 [2:27:00<50:36,  5.90s/it]


 74%|███████▍  | 1497/2011 [2:27:05<50:30,  5.90s/it]


 74%|███████▍  | 1498/2011 [2:27:11<50:24,  5.90s/it]


 75%|███████▍  | 1499/2011 [2:27:17<50:18,  5.90s/it]


 75%|███████▍  | 1500/2011 [2:27:23<50:12,  5.90s/it]


 75%|███████▍  | 1501/2011 [2:27:29<50:06,  5.90s/it]


 75%|███████▍  | 1502/2011 [2:27:35<50:00,  5.90s/it]


 75%|███████▍  | 1503/2011 [2:27:41<49:54,  5.90s/it]


 75%|███████▍  | 1504/2011 [2:27:46<49:49,  5.90s/it]


 75%|███████▍  | 1505/2011 [2:27:52<49:43,  5.90s/it]


 75%|███████▍  | 1506/2011 [2:27:58<49:37,  5.90s/it]


 75%|███████▍  | 1507/2011 [2:28:04<49:31,  5.90s/it]


 75%|███████▍  | 1508/2011 [2:28:10<49:25,  5.90s/it]


 75%|███████▌  | 1509/2011 [2:28:16<49:19,  5.90s/it]


 75%|███████▌  | 1510/2011 [2:28:21<49:13,  5.90s/it]


 75%|███████▌  | 1511/2011 [2:28:27<49:07,  5.90s/it]


 75%|███████▌  | 1512/2011 [2:28:33<49:01,  5.90s/it]


 75%|███████▌  | 1513/2011 [2:28:39<48:55,  5.90

 81%|████████  | 1622/2011 [2:39:24<38:13,  5.90s/it]


 81%|████████  | 1623/2011 [2:39:30<38:07,  5.90s/it]


 81%|████████  | 1624/2011 [2:39:35<38:01,  5.90s/it]


 81%|████████  | 1625/2011 [2:39:41<37:56,  5.90s/it]


 81%|████████  | 1626/2011 [2:39:47<37:50,  5.90s/it]


 81%|████████  | 1627/2011 [2:39:53<37:44,  5.90s/it]


 81%|████████  | 1628/2011 [2:39:59<37:38,  5.90s/it]


 81%|████████  | 1629/2011 [2:40:05<37:32,  5.90s/it]


 81%|████████  | 1630/2011 [2:40:11<37:26,  5.90s/it]


 81%|████████  | 1631/2011 [2:40:17<37:20,  5.90s/it]


 81%|████████  | 1632/2011 [2:40:22<37:14,  5.90s/it]


 81%|████████  | 1633/2011 [2:40:28<37:08,  5.90s/it]


 81%|████████▏ | 1634/2011 [2:40:34<37:02,  5.90s/it]


 81%|████████▏ | 1635/2011 [2:40:40<36:57,  5.90s/it]


 81%|████████▏ | 1636/2011 [2:40:46<36:51,  5.90s/it]


 81%|████████▏ | 1637/2011 [2:40:52<36:45,  5.90s/it]


 81%|████████▏ | 1638/2011 [2:40:58<36:39,  5.90s/it]


 82%|████████▏ | 1639/2011 [2:41:04<36:33,  5.90

 87%|████████▋ | 1748/2011 [2:51:47<25:50,  5.90s/it]


 87%|████████▋ | 1749/2011 [2:51:53<25:44,  5.90s/it]


 87%|████████▋ | 1750/2011 [2:51:59<25:39,  5.90s/it]


 87%|████████▋ | 1751/2011 [2:52:05<25:33,  5.90s/it]


 87%|████████▋ | 1752/2011 [2:52:11<25:27,  5.90s/it]


 87%|████████▋ | 1753/2011 [2:52:17<25:21,  5.90s/it]


 87%|████████▋ | 1754/2011 [2:52:22<25:15,  5.90s/it]


 87%|████████▋ | 1755/2011 [2:52:28<25:09,  5.90s/it]


 87%|████████▋ | 1756/2011 [2:52:34<25:03,  5.90s/it]


 87%|████████▋ | 1757/2011 [2:52:40<24:57,  5.90s/it]


 87%|████████▋ | 1758/2011 [2:52:46<24:51,  5.90s/it]


 87%|████████▋ | 1759/2011 [2:52:52<24:45,  5.90s/it]


 88%|████████▊ | 1760/2011 [2:52:58<24:40,  5.90s/it]


 88%|████████▊ | 1761/2011 [2:53:04<24:34,  5.90s/it]


 88%|████████▊ | 1762/2011 [2:53:09<24:28,  5.90s/it]


 88%|████████▊ | 1763/2011 [2:53:15<24:22,  5.90s/it]


 88%|████████▊ | 1764/2011 [2:53:21<24:16,  5.90s/it]


 88%|████████▊ | 1765/2011 [2:53:27<24:10,  5.90

 93%|█████████▎| 1874/2011 [3:04:10<13:27,  5.90s/it]


 93%|█████████▎| 1875/2011 [3:04:16<13:21,  5.90s/it]


 93%|█████████▎| 1876/2011 [3:04:22<13:16,  5.90s/it]


 93%|█████████▎| 1877/2011 [3:04:28<13:10,  5.90s/it]


 93%|█████████▎| 1878/2011 [3:04:34<13:04,  5.90s/it]


 93%|█████████▎| 1879/2011 [3:04:40<12:58,  5.90s/it]


 93%|█████████▎| 1880/2011 [3:04:46<12:52,  5.90s/it]


 94%|█████████▎| 1881/2011 [3:04:52<12:46,  5.90s/it]


 94%|█████████▎| 1882/2011 [3:04:58<12:40,  5.90s/it]


 94%|█████████▎| 1883/2011 [3:05:03<12:34,  5.90s/it]


 94%|█████████▎| 1884/2011 [3:05:09<12:28,  5.90s/it]


 94%|█████████▎| 1885/2011 [3:05:15<12:23,  5.90s/it]


 94%|█████████▍| 1886/2011 [3:05:21<12:17,  5.90s/it]


 94%|█████████▍| 1887/2011 [3:05:27<12:11,  5.90s/it]


 94%|█████████▍| 1888/2011 [3:05:33<12:05,  5.90s/it]


 94%|█████████▍| 1889/2011 [3:05:39<11:59,  5.90s/it]


 94%|█████████▍| 1890/2011 [3:05:45<11:53,  5.90s/it]


 94%|█████████▍| 1891/2011 [3:05:51<11:47,  5.90

 99%|█████████▉| 2000/2011 [3:16:36<01:04,  5.90s/it]


100%|█████████▉| 2001/2011 [3:16:42<00:58,  5.90s/it]


100%|█████████▉| 2002/2011 [3:16:48<00:53,  5.90s/it]


100%|█████████▉| 2003/2011 [3:16:54<00:47,  5.90s/it]


100%|█████████▉| 2004/2011 [3:17:00<00:41,  5.90s/it]


100%|█████████▉| 2005/2011 [3:17:05<00:35,  5.90s/it]


100%|█████████▉| 2006/2011 [3:17:11<00:29,  5.90s/it]


100%|█████████▉| 2007/2011 [3:17:17<00:23,  5.90s/it]


100%|█████████▉| 2008/2011 [3:17:23<00:17,  5.90s/it]


100%|█████████▉| 2009/2011 [3:17:29<00:11,  5.90s/it]


100%|█████████▉| 2010/2011 [3:17:35<00:05,  5.90s/it]


100%|██████████| 2011/2011 [3:17:41<00:00,  5.90s/it]




In [42]:
from tqdm import tqdm

tfidf_nearest_features = np.vstack(tfidf_features)
del tfidf_features, prices, train_prices, train_targets

In [43]:
with open("_tmp_nearest_tfidf_features.pickle", "wb") as f:
    pickle.dump(tfidf_nearest_features, f)

In [44]:
with open("_tmp_nearest_tfidf_features.pickle", "rb") as f:
    tfidf_nearest_features = pickle.load(f)
tfidf_nearest_features = pd.DataFrame(tfidf_nearest_features, columns = ["tfidf_" + x for x in feature_names],
                                    index=df.index)
tfidf_nearest_features.to_csv("tf_idf_nearest_features.csv.zip", compression="zip")

In [46]:
del tfidf_nearest_features
gc.collect()

1226754

# Categorial features

In [ ]:
cat_img_nearest_features = {}

In [ ]:
for cat in np.unique(df_with_image["category_name"]):
    print(cat)
    
    img_features = []
    if len(cat_img_subset_train[cat]) < np.max(ks):
        continue
    
    nearest_objects = cat_img_indices[cat].query(np.array(cat_img_subset[cat][dense_img_vec_names]),
                                                 k=np.max(ks))
    
    prices = cat_img_subset[cat]["price"]
    targets_train = y.loc[cat_img_subset_train[cat].index]
    
    for i, (dists, indices) in enumerate(zip(*nearest_objects)):
        f = get_features(i, indices, dists, cat_img_subset_train[cat]["price"], targets_train, prices)
        img_features.append(f)
    img_features = np.vstack(img_features)
    cat_img_nearest_features[cat] = img_features

In [ ]:
with open("_tmp_nearest_cat_img_features.pickle", "wb") as f:
    pickle.dump(cat_img_nearest_features, f)